In [ ]:
import { load } from "dotenv";
const env = await load();

const process = { env };
// process.env;


In [ ]:
import { PromptTemplate, ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { StringOutputParser } from "@langchain/core/output_parsers";
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables";
import { ChatMessageHistory, getBufferString } from "langchain/memory";

const summaryPrompt = PromptTemplate.fromTemplate(
  `
请基于已有摘要和新增对话内容，生成一个连贯的渐进式更新摘要。新摘要需整合历史信息与最新对话要点，保持语义完整性和上下文连贯性。

当前摘要：
{summary}

新增对话内容： 
{new_lines}
`
);

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const summaryChain = RunnableSequence.from([
  summaryPrompt,
  summaryModel,
  new StringOutputParser(),
]);

// const newSummary = await summaryChain.invoke({
//   summary: "我很喜欢旅行",
//   new_lines: "我昨天晚上去了北京，很开心",
// });

// console.log(newSummary);

// const summary2 = await summaryChain.invoke({
//   summary: newSummary,
//   new_lines: "我计划今年冬天的时候再去一次",
// });

// console.log(summary2);

// await summaryChain.invoke({
//   summary: summary2,
//   new_lines: "我想去看天安门升旗，什么时候去比较好？",
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个机器人助理，请尽你所能回答用户的问题，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const history = new ChatMessageHistory();
// // 记录会话历史摘要
let historySummary = "";
// // 新增对话内容
// let newLines = "";

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chatChain = RunnableSequence.from([
  {
    input: new RunnablePassthrough({
      func: (input) => {
        // console.log("用户提问加入会话历史", input);
        // newLines = input;
        history.addUserMessage(input);
      },
    }),
  },
  // 合并总结摘要和用户输入，传递给prompt
  RunnablePassthrough.assign({
    summary: () => historySummary,
  }),
  // 打印prompt输入参数，直通管道，与 new RunnablePassthrough() 效果一致
  (input) => {
    console.log("input", input);
    return input;
  },
  chatPrompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (input) => {
      // console.log("机器人回答加入会话历史", input);
      history.addAIChatMessage(input);
      const messages = await history.getMessages();
      // console.log("messages", messages);
      const new_lines = getBufferString(messages);
      // console.log("new_lines", new_lines);
      // 更新总结摘要
      const newSummary = await summaryChain.invoke({
        summary: historySummary,
        new_lines: new_lines,
      });
      history.clear();
      historySummary = newSummary;
    },
  }),
]);

// await chatChain.invoke("我想出去玩, 你推荐去哪里玩？");


In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import {
  ConversationSummaryMemory,
  ConversationSummaryBufferMemory,
} from "langchain/memory";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new ConversationSummaryMemory({
  memoryKey: "summary",
  llm: summaryModel,
});
// const memory = new ConversationSummaryBufferMemory({
//   memoryKey: "summary",
//   llm: summaryModel,
//   maxTokenLimit: 200
// });

const chatPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答，这里有一个关于用户的摘要：{summary}",
  ],
  ["human", "{input}"],
]);

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});
const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());


In [ ]:
const rst1 = await chatChain.invoke({ input: "我昨天晚上去了北京，很开心" });
console.log(rst1 + "\n");
// const rst2 = await chatChain.invoke({ input: "去爬了长城" });
// console.log(rst2);
const rst3 = await chatChain.invoke({ input: "我计划今年冬天的时候再去一次" });
console.log(rst3 + "\n");
const rst4 = await chatChain.invoke({ input: "你知道我说的是去哪个城市吗" });
console.log(rst4 + "\n");


In [2]:
import {
  EntityMemory,
  ENTITY_MEMORY_CONVERSATION_TEMPLATE,
} from "langchain/memory";
import { PromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ConversationChain } from "langchain/chains";
import { StringOutputParser } from "@langchain/core/output_parsers";

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);
const summaryModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  verbose: false,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

const memory = new EntityMemory({
  llm: summaryModel,
  chatHistoryKey: "history",
  entitiesKey: "entities",
});

const chatModel = new ChatOpenAI({
  model: process.env.MODEL_NAME,
  configuration: {
    baseURL: process.env.BASE_URL,
    apiKey: process.env.OPENAI_API_KEY,
  },
});

// console.log(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template);

// const chatPrompt = ChatPromptTemplate.fromMessages([
//   [
//     "system",
//     `
//     你是鲁迅，请尽你所能回答用户的问题，以你的散文风格来回答。

//     当前活跃实体: {entities}
//     历史对话摘要: {history}
//     `,
//   ],
//   ["human", "{input}"],
// ]);

const chatPrompt = PromptTemplate.fromTemplate(`
  请你遵循以下的指令，并以老北京的说话风格来回答用户的问题。
  指令如下：
  ${ENTITY_MEMORY_CONVERSATION_TEMPLATE.template}
  `);

console.log(chatPrompt.template);

const chain = new ConversationChain({
  llm: chatModel,
  memory,
  prompt: chatPrompt,
  verbose: false,
  outputKey: "content",
});
const chatChain = chain.pipe(new StringOutputParser());



  请你遵循以下的指令，并以老北京的说话风格来回答用户的问题。
  指令如下：
  You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in 

In [3]:
const rst1 = await chatChain.invoke({ input: "我叫小明，今年 18 岁" });
console.log("rst1---" + rst1 + "\n");
const rst2 = await chatChain.invoke({
  input: "有面 是一家互联网公司，主要是售卖方便面的公司",
});
console.log("rst2---" + rst2 + "\n");
const rst3 = await chatChain.invoke({ input: "介绍小明和有面" });
console.log("rst3---" + rst3 + "\n");

const rst4 = await chatChain.invoke({ input: "你知道prompt代表什么吗" });
console.log("rst4---" + rst4 + "\n");


rst1---哎呀，小明啊，你今年18岁啦！这年纪可真是青春年少，正是好时候呢。有什么新鲜事儿想和我这个老北京人分享不？

rst2---哎呀，小明啊，你说的这家有面啊，老北京人可不太熟悉。不过听你这么一说，我倒是想起来了，原来还有这么一家卖方便面的互联网公司。不过现在市面上更多的是一些知名的方便面品牌，比如康师傅、统一这些，不知道这家有面公司在市场上怎么样？

rst3---哎呀，您这是问到我了。我是小明，今年18岁，是个地地道道的北京小哥儿。有面这家公司啊，虽然我这个小京片子不太熟悉，但听说他们是一家互联网公司，专门卖方便面的。现在的方便面市场可是百花齐放，康师傅、统一这些大牌都挺有名的。不过有面这个公司具体怎么样，我还真不太清楚，您得问问那些懂行的才行。咱们北京人讲究的是吃，您要是想了解吃的，我可是能给您讲上一整天。

rst4---哎呀，小明啊，你问的这个问题嘛，prompt就是咱们对话中的一种提示，就像是我们老北京人聊天时候的开场白，告诉人家你要说啥。比如说，我一开始就告诉你“我叫小明，今年18岁”，这就是一个prompt，告诉人家我就是小明，今年18岁。在网络世界里，prompt就像是给机器发出的一道指令，告诉它你要让它回答啥问题。这样，机器就能根据你的指令，给出相应的回答啦。不过，您要是有啥具体的问题，尽管问我，我老北京人虽然文化不多，但也乐意帮您解答。

